# A step in open channel flow

by Xiaofeng Liu, Ph.D., P.E.
Associate Professor

Department of Civil and Environmental Engineering

Institute of Computational and Data Sciences

Penn State University

223B Sackett Building, University Park, PA 16802

Web: http://water.engr.psu.edu/liu/

-------------------------------------------------

In [2]:
#Do not change this cell. It is for setting things up.

#make plots inline
%matplotlib inline

#import the preamble in the parent directory
import sys
sys.path.append("../../")
from preamble import *

In [3]:
%%javascript
MathJax.Hub.Config({
    TeX: { equationNumbers: { autoNumber: "AMS" } }
});

<IPython.core.display.Javascript object>

## Introduction

In open channels, such as rivers and streams, when a step is installed, the hydraulics of the channel at the step will be changed. An example of a step in open channels is a sill used for erosion control or prevent the propogation of salt wedge from the ocean to the upstream of a river. We can use simple conservation laws to analyze the hydraulics. In its simnple form, the concept of specific energy can be used. 

The problem is defined as in the following figure. The channel has uniform width $b$ and the step height is $\Delta z$. The incoming flow at cross section 1 has a velocity of $V_1$ and depth of $y_1$. An important engineering question is what the water depth $y_2$ and velocity $V_2$ at the step. In this example, we will not go to the details of the hydraulics theory. Interested readers can find sucn information in any open channel hydraulics textbook. 

We will apply both conservation of mass and energy. First, based on conservation of mass between cross sections 1 and 2, we can write
\begin{equation}
V_1 y_1 b = V_2 y_2 b
\end{equation}
Also, from conservation of energy (ignore any energy loss), we have
\begin{equation}
y_1 + \frac{V_1^2}{2g} = y_2 + \frac{V_2^2}{2g} + \Delta Z
\end{equation}
where $\Delta Z$ is height of the step. In the previous two equations, we only have two unknowns, i.e., $y_2$ and $V_2$. Everything else are given. So the problem can be solved. Solve for $y_2$, we can get the following equation:
\begin{equation}
y_2^3 - \left[ y_1 + \frac{V_1^2}{2g}-\Delta z \right] y_2^2 + \frac{V_1^2 y_1^2}{2g} = 0
\end{equation}
This is a third-order polynomial for $y_2$ and it can be solved numerically. In fact, if the step height $\Delta z$ is not too large, there are three roots, of which two are positive. In real world, we can only have one water depth at the step. So the selection of the positive root depends on whether the flow subcritical or critical. If it subcritical, pick the large positive root; otherwise, pick the smaller positive root. The specific energy diagram below clearly shows this. In the figure, we assume the flow is subcritical such that we are on the upper branch of the specific energy curve. With a step, the point (cross-section 1) on the curve will move down the upper curve to the left and thus the water depth will decrease at the step (cross-section 2).

<img src="step_in_OCF.png" width="600"/>
<h3 align="center">Figure. A step in an open channel.</h3> 

In the following, we will implement the above analysis with Python code. We assume the step height $\Delta z$ is sufficiently small. 

In [25]:
from scipy import optimize
import numpy as np

#define the function
def f(y2,y1,V1,deltaZ):
    return y2**3-(y1+V1**2/2.0/9.8-deltaZ)*y2**2+V1**2*y1**2/2.0/9.8
         
    
#define parameters for the problem
y1 = 1  #water depth at cross section 1
V1 = 0.2  #mean velocity at cross section 1
deltaZ = 0.1 #step height
    
#directly call SciPy optimize's newton method
y2=optimize.newton(f,  y1, args=(y1,V1,deltaZ,))      # starting point x0 = 2

#print out the solution
print("The Froude number at cross section 1 = ", round(V1/np.sqrt(9.8*y1),2))
print("Water depth at the step y2 = ", round(y2,4), " m.")

print("Done!")


The Froude number at cross section 1 =  0.06
Water depth at the step y2 =  0.8995  m.
Done!


The above analysis and code assume the step size $\Delta z$ is not that large. But how large is large? What if the step gets really large? In reality, if $\Delta z$ is larger than certain threshold $\Delta z_{max}$, the upstream flow will be "backed up", thereby increasing its specific energy from $E_1$ to $E_1^*$. This situation is also called "choking". The new $E_1^*$ will be just enough to accomndate the bump, i.e., the flow over the bump will be exactly critical ($y_2=y_c$). At critical flow condition, $E_2=E_{min}$. Thus, $E_1^*=E_{min}+\Delta z$. For a rectangular channel, the mininum specific energy can be calcuated as 
\begin{equation}
E_{min} = \frac{3}{2} \left(\frac{q^2}{g} \right)^{1/3}
\end{equation}
where $q$ is the specific discharge defined as $Vy$. For a rectangular channel with constant width, the specific discharge does not change along the channel due to mass conservation. 

At the threshold condition, the maximum step height is then 
\begin{equation}
\Delta z_{max} = E_1 - E_{min}
\end{equation}

So for a given inflow condition, the $\Delta z_{max}$ is firstl calculated and whether the step will choke the flow will checked. If not, the calculation will be similar as the previous example. If choked, then the flow status at the first cross section will be adjusted and the the flow is critical the step. The adjusted the flow depth and velocity can be calculated as follows. The adjusted specific energy at cross section 1 is 
\begin{equation}
E_1^* = E_{min} + \Delta z
\end{equation}
From definition, we also have
\begin{equation}
y_1^* + \frac{V_1^{*2}}{2g} = E_1^*
\end{equation}
The adjusted upstream flow still obeys the mass of conservatino law, thus
\begin{equation}
V_1^*y_1^* = V_1 y_1
\end{equation}
In the previous two equations, the only two unknonws are $V_1^*$ and $y_1^*$. As before, we can eleminate one of the two unknowns and sovle for the other. As a result, we can solve for $y_1^*$ by finding the roots of the following equation
\begin{equation}
y_1^{*3} - E_1^* y_1^{*,2} + \frac{V_1^2 y_1^2}{2g} = 0
\end{equation}



The following Python code implements the algorithm described above. 

In [43]:
from scipy import optimize
import numpy as np

#define the function if not choked
def f(y2,y1,V1,deltaZ):
    return y2**3-(y1+V1**2/2.0/9.8-deltaZ)*y2**2+V1**2*y1**2/2.0/9.8

#define the function if choked
def f_choked(y1star,y1,V1,E1star):
    return y1star**3-E1star*y1star**2+V1**2*y1**2/2.0/9.8

    
#define parameters for the problem
y1 = 0.9  #water depth at cross section 1
V1 = 0.75  #mean velocity at cross section 1
deltaZ = 0.5 #step height

#specific discharge
q = y1*V1

#critcial depth
yc = (q**2/9.8)**(1.0/3.0)

#calcuate E_min
Emin=3.0/2.0*(q**2/9.8)**(1.0/3.0)
#print("Emin = ", Emin)

#calculate E1
E1 = y1 + V1**2/2/9.8
#print("E1 = ", E1)

#calculate deltaZmax
deltaZmax = E1 - Emin

print("deltaZmax,deltaZ = ", round(deltaZmax,3),deltaZ)

#check choking or not
if deltaZ < deltaZmax: 
    print("Flow is not choked.")
    #directly call SciPy optimize's newton method
    y2=optimize.newton(f,  y1, args=(y1,V1,deltaZ,))      # starting point x0 = 2

    #print out the solution
    print("The Froude number at cross section 1 = ", round(V1/np.sqrt(9.8*y1),2))
    print("Water depth at the step y2 = ", round(y2,4), " m.")
    
else:
    print("Flow is choked.")
    
    #flow at the step is critical
    y2=yc
    
    #new specific energy at the upstream cross section 1
    E1star = Emin + deltaZ
    #print("E1star = ", E1star)    
    
    #call SciPy optimize's newton method
    y1star=optimize.newton(f_choked, 1, args=(y1,V1,E1star))
    
    #print out the solution
    print("The Froude number at cross section 1 = ", round(V1/np.sqrt(9.8*y1),2))
    print("Water depth at the step y2 = ", round(y2,4), " m.")
    print("New water depth at section 1 is y1star = ", round(y1star,4), " m.")


print("Done!")

deltaZmax,deltaZ =  0.389 0.5
Flow is choked.
The Froude number at cross section 1 =  0.25
Water depth at the step y2 =  0.3596  m.
New water depth at section 1 is y1star =  1.0169  m.
Done!
